In [1]:

#Schrodinger on 3bulb using bunimovich contour and complex shift. This code allows for LC of ev. Not as up to date as other codes though.

#Import the other code
# import dirichlet1D

#from pyeigfeast.spectralproj.mat import Vecs, SpectralProjMat

# For feast implementation:
from pyeigfeast.spectralproj.ngs import NGvecs, SpectralProjNG
from netgen.geom2d import SplineGeometry
import numpy as np
import ngsolve as ngs
import scipy.sparse as sp

from ngsolve.webgui import Draw

import scipy.linalg   #LApack for linear solver at end
from ngsolve import grad, dx

# For direct eigenvalue computation by scipy:
from scipy.sparse.linalg import eigs
from scipy.sparse import coo_matrix
from scipy.linalg import norm


def make_threebulb_left_sub(h, S1_x, S1_y):     #this is to segment left bulb

   S1_xcoords, S1_ycoords = [], []
   ind1_x, ind1_y = 0.2, 0.2
   while ind1_x <=3.8:
       S1_xcoords.append(ind1_x)
       ind1_x += 3.6/S1_x
   while ind1_y <=3.8:
       S1_ycoords.append(ind1_y)
       ind1_y += 3.6/S1_y
   print("S1_y", S1_ycoords)
   ds = 0.1
   geometry = SplineGeometry()
   for i in range(S1_x):
       if i < S1_x:
           for j in range(S1_y):
               if j < S1_y:
                   geometry.AddRectangle((S1_xcoords[i]+i*ds,S1_ycoords[j]+j*ds), (S1_xcoords[i+1]+i*ds,S1_ycoords[j+1]+j*ds),
                   bcs=['b{}{}'.format(i,j),'r{}{}'.format(i,j),'t{}{}'.format(i,j),'l{}{}'.format(i,j)], leftdomain=1, rightdomain=1)
   pntlist = [(0, 0), (13, 0), (13, 3), (10, 3), (10, 1),
        (8, 1), (8, 2), (6, 2), (6, 1), (4, 1), (4, 4), (0, 4), (4, 0)]
   pnts = [geometry.AppendPoint(*p) for p in pntlist]
   seglist = [(pnts[0], pnts[12], 'allbdry', 1, 0),
           (pnts[12], pnts[9], ' ', 1, 2),
           (pnts[9], pnts[10], 'allbdry', 1, 0),
           (pnts[10], pnts[11], 'allbdry', 1, 0),
           (pnts[11], pnts[0], 'allbdry', 1, 0),
           (pnts[12], pnts[1], 'allbdry', 2, 0),
           (pnts[1], pnts[2], 'allbdry', 2, 0),
           (pnts[2], pnts[3], 'allbdry', 2, 0),
           (pnts[3], pnts[4], 'allbdry', 2, 0),
           (pnts[4], pnts[5], 'allbdry', 2, 0),
           (pnts[5], pnts[6], 'allbdry', 2, 0),
           (pnts[6], pnts[7], 'allbdry', 2, 0),
           (pnts[7], pnts[8], 'allbdry', 2, 0),
           (pnts[8], pnts[9], 'allbdry', 2, 0)]
   for p0, p1, bn, ml, mr in seglist:
    geometry.Append(['line', p0, p1],
            bc=bn, leftdomain=ml, rightdomain=mr)
   tmp, kth = 0, []
   for i in range(S1_x):
       for j in range(S1_y):
           tmp +=1
           print('tmp', tmp)
           kth.append(tmp)  
           geometry.SetMaterial(tmp, 'sub{}{}'.format(i,j))
   main_material_num = kth[-1]+1 
   geometry.SetMaterial(main_material_num, 'domain')
   m_sub = geometry.GenerateMesh(maxh=h)
   return m_sub

def eig_fem_feast(mesh, p, m, ctr, rad, npts, S1_x, S1_y, verbose=True, quadrule='circ_trapez_shift', check_contour=2, stop_tol=1.e-9,
                reduce_by_symmetry=True, rhoinv=0, Y=None, niterations=50, inverse=None):

    domain_values = {}
    values_list = []
    for i in range(S1_x):
        for j in range(S1_y):
            domain_values['sub{}{}'.format(i,j)] =  10*np.random.rand(1)
            #values_list.append(list(domain_values.values()))
            #values_list = [domain_values[mat] for mat in mesh.GetMaterials()]
    domain_values['domain'] =  0
    print("dom val", domain_values)
    #values_list = np.fromiter(domain_values.values(), dtype=float)
    values_list.append(list(domain_values.values()))
    print("values_list =", values_list)
    
    cf = ngs.CoefficientFunction(values_list)
    ngs.ngsglobals.msg_level = 1
    X = ngs.H1(mesh, order=p, dirichlet='allbdry', complex=True)
    gfu = ngs.GridFunction(X)
    gfu.Set(cf)
    u, v = X.TnT()
    a = ngs.BilinearForm(X)
    a += ngs.grad(u) * ngs.grad(v) * dx + gfu*u*v*dx
    b = ngs.BilinearForm(X)
    b += u * v * dx
    a.Assemble()
    b.Assemble()
    P = SpectralProjNG(X, a.mat, b.mat, radius=rad, center=ctr, npts=npts,
                              reduce_sym=reduce_by_symmetry, verbose=verbose, inverse=inverse)
    if Y is None:
        Y = NGvecs(X, m, b.mat)
        Y.setrandom()
        lam, Y, history, _ = P.feast(Y, check_contour=check_contour,
                              stop_tol=stop_tol, niterations=niterations)
#     Y.draw()

    return lam, Y, history, P, a, b, X, gfu, cf




importing NGSolve-6.2.2203-56-g47f483033


In [6]:
    # PARAMETERS:     
    ctr = 5   
    npts = 4
    p = 1
    m = 10                     # initial number of vectors
    h1 = 0.1    #mesh size parameter   
 
    S1_x = 2  #how many subsquares in first square in x,y direction 
    S1_y = 2
    mesh = ngs.Mesh(make_threebulb_left_sub(h1, S1_x, S1_y))

    rad = 5
    lam, Y, history, P, A, B, X, gfu, cf = eig_fem_feast(mesh, p, m, ctr, rad, npts, S1_x, S1_y,
                                                        niterations=10) 


S1_y [0.2, 2.0, 3.8]
tmp 1
tmp 2
tmp 3
tmp 4
 Generate Mesh from spline geometry
dom val {'sub00': array([3.54617228]), 'sub01': array([0.99694236]), 'sub10': array([5.72682388]), 'sub11': array([4.81952226]), 'domain': 0}
values_list = [[array([3.54617228]), array([0.99694236]), array([5.72682388]), array([4.81952226]), 0]]

SpectralProj: Setting shifted trapezoidal rule quadrature on circular contour
SpectralProj: Radius=5, Center=5+0j

SpectralProjNG: Computing resolvents along the contour using
                inverse type umfpack
SpectralProjNG:   Factorizing at z =  +8.536 +3.536j
SpectralProjNG:   Factorizing at z =  +1.464 +3.536j

=========== Starting FEAST iterations ===========
Trying with 10 vectors:

 ITERATION 1 with 10 vectors
   Real part of computed eigenvalues:
   [ 3.09619695  4.96241509  5.31337187  6.01634255  7.29813058  8.06334544
  8.77480319  9.33812652  9.83209833 10.5631794 ]
   Relative Hausdorff distance from prior iterate: 2.000e+99

 ITERATION 2 with 10 v

In [9]:
Draw(Y.gridfun(), mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene